In [1]:
import nlp_basictasks
import os,json
import numpy as np
import torch
import torch.nn as nn
import random
from tqdm.autonotebook import tqdm, trange
from torch.utils.data import DataLoader
from nlp_basictasks.modules import SBERT
from nlp_basictasks.modules.transformers import BertTokenizer
from nlp_basictasks.readers.sts import InputExample,convert_examples_to_features,getExamples,convert_sentences_to_features
from nlp_basictasks.modules.utils import get_optimizer,get_scheduler
from nlp_basictasks.Trainer import Trainer
from nlp_basictasks.evaluation import stsEvaluator
from sentence_transformers import SentenceTransformer
model_path1='/data/nfs14/nfs/aisearch/asr/xhsun/bwbd_recall/distill-simcse/'
model_path2="/data/nfs14/nfs/aisearch/asr/xhsun/bwbd_recall/distiluse-base-multilingual-cased-v1/"
# data_folder='/data/nfs14/nfs/aisearch/asr/xhsun/datasets/lcqmc/'
# train_file=os.path.join(data_folder,'lcqmc_train.tsv')
# dev_file=os.path.join(data_folder,'lcqmc_dev.tsv')
data_folder='/data/nfs14/nfs/aisearch/asr/xhsun/bwbd_recall/ssc_data/sbert_ce_portion_5'
train_file=os.path.join(data_folder,'train.txt')
dev_file=os.path.join(data_folder,'dev.txt')
tokenizer=BertTokenizer.from_pretrained(os.path.join(model_path1,'0_Transformer'))
max_seq_len=64
batch_size=128

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


2021-10-12 18:26:30 - INFO - <module> - 54 : Loading faiss with AVX2 support.
2021-10-12 18:26:30 - INFO - <module> - 58 : Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2021-10-12 18:26:30 - INFO - <module> - 64 : Loading faiss.
2021-10-12 18:26:30 - INFO - <module> - 66 : Successfully loaded faiss.
2021-10-12 18:26:30 - INFO - from_pretrained - 125 : loading vocabulary file /data/nfs14/nfs/aisearch/asr/xhsun/bwbd_recall/distill-simcse/0_Transformer/vocab.txt


# 定义模型

In [18]:
class SimCSE(nn.Module):
    def __init__(self,
                 bert_model_path,
                temperature=0.05,
                is_distilbert=False,
                device='cpu'):
        super(SimCSE,self).__init__()
        self.encoder=SentenceTransformer(model_name_or_path=bert_model_path,device=device)
        self.temperature=temperature
        self.is_distilbert=is_distilbert#蒸馏版本的BERT不支持token_type_ids
    def cal_cos_sim(self,embeddings1,embeddings2):
        embeddings1_norm=torch.nn.functional.normalize(embeddings1,p=2,dim=1)
        embeddings2_norm=torch.nn.functional.normalize(embeddings2,p=2,dim=1)
        return torch.mm(embeddings1_norm,embeddings2_norm.transpose(0,1))#(batch_size,batch_size)
        
    def forward(self,batch_inputs):
        '''
        为了实现兼容，所有model的batch_inputs最后一个位置必须是labels，即使为None
        get token_embeddings,cls_token_embeddings,sentence_embeddings
        sentence_embeddings是经过Pooling层后concat的embedding。维度=768*k，其中k取决于pooling的策略
        一般来讲，只会取一种pooling策略，要么直接cls要么mean last or mean last2 or mean first and last layer，所以sentence_embeddings的维度也是768
        '''
        batch1_features,batch2_features,_=batch_inputs
        if self.is_distilbert:
            del batch1_features['token_type_ids']
            del batch2_features['token_type_ids']
        batch1_embeddings=self.encoder(batch1_features)['sentence_embedding']
        batch2_embeddings=self.encoder(batch2_features)['sentence_embedding']
        cos_sim=self.cal_cos_sim(batch1_embeddings,batch2_embeddings)/self.temperature#(batch_size,batch_size)
        batch_size=cos_sim.size(0)
        assert cos_sim.size()==(batch_size,batch_size)
        labels=torch.arange(batch_size).to(cos_sim.device)
        return nn.CrossEntropyLoss()(cos_sim,labels)
    
    def encode(self, sentences,
               batch_size: int = 32,
               show_progress_bar: bool = None,
               output_value: str = 'sentence_embedding',
               convert_to_numpy: bool = True,
               convert_to_tensor: bool = False,
               device: str = None,
               normalize_embeddings: bool = False):
        '''
        传进来的sentences只能是single_batch
        '''
        return self.encoder.encode(sentences=sentences,
                                         batch_size=batch_size,
                                         show_progress_bar=show_progress_bar,
                                         output_value=output_value,
                                         convert_to_numpy=convert_to_numpy,
                                         convert_to_tensor=convert_to_tensor,
                                         device=device,
                                         normalize_embeddings=normalize_embeddings)
    
    def save(self,output_path):
        os.makedirs(output_path,exist_ok=True)
        with open(os.path.join(output_path, 'model_param_config.json'), 'w') as fOut:
            json.dump(self.get_config_dict(output_path), fOut)
        self.encoder.save(output_path)
        
    def get_config_dict(self,output_path):
        '''
        一定要有dict，这样才能初始化Model
        '''
        return {'output_path':output_path,'temperature': self.temperature, 'is_distilbert': self.is_distilbert}
    @staticmethod
    def load(input_path):
        with open(os.path.join(input_path, 'model_param_config.json')) as fIn:
            config = json.load(fIn)
        return SimCSE(**config)

In [19]:
device='cpu'
simcse=SimCSE(bert_model_path=model_path2,is_distilbert=True,device=device)

2021-10-12 18:28:42 - INFO - __init__ - 41 : Load pretrained SentenceTransformer: /data/nfs14/nfs/aisearch/asr/xhsun/bwbd_recall/distiluse-base-multilingual-cased-v1/
2021-10-12 18:28:42 - INFO - __init__ - 107 : Load SentenceTransformer from folder: /data/nfs14/nfs/aisearch/asr/xhsun/bwbd_recall/distiluse-base-multilingual-cased-v1/


# 获取 dataloader

## 获取train_dataloader

In [21]:
train_sentences=[]
with open(train_file) as f:
    lines=f.readlines()
    for line in lines:
        line_split=line.strip().split('\t')
        if line_split[-1]=='1':
            train_sentences.append([line_split[0],line_split[1]])
#train_sentences=list(train_sentences)
print(len(train_sentences))

144396


In [22]:
train_sentences[:3]

[['资金监管怎么弄', '资金监管卖方怎么操作？'], ['升的积分', 'm8对应积分是多少'], ['换绑定号码', '更改绑定手机号']]

In [23]:
print(train_sentences[:2])
random.shuffle(train_sentences)
train_sentences=train_sentences
print(len(train_sentences))
print(train_sentences[:2])

[['资金监管怎么弄', '资金监管卖方怎么操作？'], ['升的积分', 'm8对应积分是多少']]
144396
[['怎么改房源标签', '房源标签有哪些？'], ['二手贷款买房流程', '商业贷款流程是什么']]


In [24]:
train_examples=[InputExample(text_list=sentence,label=1) for sentence in train_sentences]
train_dataloader=DataLoader(train_examples,shuffle=True,batch_size=batch_size)
def smart_batching_collate(batch):
    features_of_a,features_of_b,labels=convert_examples_to_features(examples=batch,tokenizer=tokenizer,max_seq_len=max_seq_len)
    return features_of_a,features_of_b,labels
train_dataloader.collate_fn=smart_batching_collate

In [25]:
print(train_examples[0])

<InputExample> label: 1, text pairs : 怎么改房源标签; 房源标签有哪些？


In [26]:
convert_examples_to_features([train_examples[0]],tokenizer)

({'input_ids': tensor([[ 101, 3814, 2118, 4282, 3998, 5110, 4561]]),
  'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]),
  'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[ 101, 3998, 5110, 4561, 6087, 4461, 2869, 2159]]),
  'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]),
  'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])},
 tensor([1]))

# 获取dev_dataloader

In [27]:
dev_examples=getExamples(dev_file,label2id={"0":0,"1":1},filter_heads=True,mode='dev',isCL=False)

2021-10-12 18:29:47 - INFO - getExamples - 44 : Heads like : 首看保护期	钥匙角色上传多长时间有效	0

2021-10-12 18:29:47 - INFO - getExamples - 57 : *****************************Logging some dev examples*****************************
2021-10-12 18:29:47 - INFO - getExamples - 58 : Total dev nums is : 8946
2021-10-12 18:29:47 - INFO - getExamples - 61 : 推荐奖多久发放	房产证需要上传几页	0
2021-10-12 18:29:47 - INFO - getExamples - 61 : 门店换电脑	订购物资网站网址是什么？	1
2021-10-12 18:29:47 - INFO - getExamples - 61 : 怎么找钥匙记录	有线上签约租房的流程吗	0
2021-10-12 18:29:47 - INFO - getExamples - 61 : 经济适用房交易规定是什么	签约自如的业绩怎么查看	0
2021-10-12 18:29:47 - INFO - getExamples - 61 : 跨店东保护是什么？	学历验真流程	0


## 构造evaluator

In [28]:
dev_sentences=[example.text_list for example in dev_examples]
dev_labels=[example.label for example in dev_examples]
print(dev_sentences[0],dev_labels[0])
sentences1_list=[sen[0] for sen in dev_sentences]
sentences2_list=[sen[1] for sen in dev_sentences]
evaluator=stsEvaluator(sentences1=sentences1_list,sentences2=sentences2_list,batch_size=64,write_csv=False,scores=dev_labels)

['啥时候发货', '来消息没声音怎么设置'] 0


In [29]:
for i in range(5):
    print(evaluator.sentences1[i],evaluator.sentences2[i],evaluator.scores[i])

啥时候发货 来消息没声音怎么设置 0
师傅如何解除 垫佣发票在哪里上传 0
我密码多少 商圈经理面访回访有次数限制吗？ 0
如何签vip 业主电话最多可以录入几个？ 0
翻录保护期 业主如何下架自己在卖的房源 0


In [30]:
evaluator(model=simcse)

2021-10-12 18:29:49 - INFO - __call__ - 72 : EmbeddingSimilarityEvaluator: Evaluating the model on  dataset:


2021-10-12 18:30:16 - INFO - __call__ - 103 : Cosine-Similarity :	Pearson: 0.7399	Spearman: 0.5806
2021-10-12 18:30:16 - INFO - __call__ - 105 : Manhattan-Distance:	Pearson: 0.7433	Spearman: 0.5800
2021-10-12 18:30:16 - INFO - __call__ - 107 : Euclidean-Distance:	Pearson: 0.7364	Spearman: 0.5756
2021-10-12 18:30:16 - INFO - __call__ - 109 : Dot-Product-Similarity:	Pearson: 0.7406	Spearman: 0.5808


0.5806207292388106

# train model

In [31]:
epochs=10
output_path='/data/nfs14/nfs/aisearch/asr/xhsun/bwbd_recall/tmp'
tensorboard_logdir=os.path.join(output_path,'log')

## 获取optimizer

In [32]:
optimizer_type='AdamW'
scheduler='WarmupLinear'
warmup_proportion=0.1
optimizer_params={'lr': 2e-5}
weight_decay=0.01
num_train_steps = int(len(train_dataloader) * epochs)
warmup_steps = num_train_steps*warmup_proportion
optimizer = get_optimizer(model=simcse,optimizer_type=optimizer_type,weight_decay=weight_decay,optimizer_params=optimizer_params)
scheduler = get_scheduler(optimizer, scheduler=scheduler, warmup_steps=warmup_steps, t_total=num_train_steps)

## 获取Trainer

In [33]:
trainer=Trainer(epochs=epochs,output_path=output_path,tensorboard_logdir=tensorboard_logdir,early_stop_patience=20)

In [34]:
trainer.train(train_dataloader=train_dataloader,
             model=simcse,
             optimizer=optimizer,
             scheduler=scheduler,
             evaluator=evaluator,
             )

2021-10-12 18:30:16 - INFO - train - 56 : 一个epoch 下，每隔225个step会输出一次loss，每隔564个step会评估一次模型


2021-10-12 18:30:19 - INFO - __call__ - 72 : EmbeddingSimilarityEvaluator: Evaluating the model on  dataset in epoch 0 after 1 steps:


2021-10-12 18:30:46 - INFO - __call__ - 103 : Cosine-Similarity :	Pearson: 0.7399	Spearman: 0.5806
2021-10-12 18:30:46 - INFO - __call__ - 105 : Manhattan-Distance:	Pearson: 0.7433	Spearman: 0.5800
2021-10-12 18:30:46 - INFO - __call__ - 107 : Euclidean-Distance:	Pearson: 0.7364	Spearman: 0.5756
2021-10-12 18:30:46 - INFO - __call__ - 109 : Dot-Product-Similarity:	Pearson: 0.7406	Spearman: 0.5808
2021-10-12 18:30:46 - INFO - save - 371 : Save model to /data/nfs14/nfs/aisearch/asr/xhsun/bwbd_recall/tmp


2021-10-12 18:30:47 - INFO - train - 98 : In epoch 0, training_step 0, the eval score is 0.5806207292388106, previous eval score is -9999999, model has been saved in /data/nfs14/nfs/aisearch/asr/xhsun/bwbd_recall/tmp
2021-10-12 18:37:12 - INFO - train - 75 : Epoch : 0, train_step : 225/11290, loss_value : 1.5816797465748258 
2021-10-12 18:43:43 - INFO - train - 75 : Epoch : 0, train_step : 450/11290, loss_value : 1.044176026450263 
2021-10-12 18:47:09 - INFO - __call__ - 72 : EmbeddingSimilarityEvaluator: Evaluating the model on  dataset in epoch 0 after 565 steps:


2021-10-12 18:47:36 - INFO - __call__ - 103 : Cosine-Similarity :	Pearson: 0.8631	Spearman: 0.6386
2021-10-12 18:47:36 - INFO - __call__ - 105 : Manhattan-Distance:	Pearson: 0.8650	Spearman: 0.6375
2021-10-12 18:47:36 - INFO - __call__ - 107 : Euclidean-Distance:	Pearson: 0.8661	Spearman: 0.6375
2021-10-12 18:47:36 - INFO - __call__ - 109 : Dot-Product-Similarity:	Pearson: 0.8513	Spearman: 0.6365
2021-10-12 18:47:36 - INFO - save - 371 : Save model to /data/nfs14/nfs/aisearch/asr/xhsun/bwbd_recall/tmp


...

2021-10-12 19:40:48 - INFO - __call__ - 103 : Cosine-Similarity :	Pearson: 0.8860	Spearman: 0.6423
2021-10-12 19:40:48 - INFO - __call__ - 105 : Manhattan-Distance:	Pearson: 0.8959	Spearman: 0.6422
2021-10-12 19:40:48 - INFO - __call__ - 107 : Euclidean-Distance:	Pearson: 0.8966	Spearman: 0.6421
2021-10-12 19:40:48 - INFO - __call__ - 109 : Dot-Product-Similarity:	Pearson: 0.8715	Spearman: 0.6405
2021-10-12 19:40:48 - INFO - save - 371 : Save model to /data/nfs14/nfs/aisearch/asr/xhsun/bwbd_recall/tmp


2021-10-12 19:40:50 - INFO - train - 98 : In epoch 2, training_step 0, the eval score is 0.6422547554728177, previous eval score is 0.6422516102367535, model has been saved in /data/nfs14/nfs/aisearch/asr/xhsun/bwbd_recall/tmp
2021-10-12 19:47:30 - INFO - train - 75 : Epoch : 2, train_step : 675/11290, loss_value : 0.46857013801733655 
2021-10-12 19:54:06 - INFO - train - 75 : Epoch : 2, train_step : 1350/11290, loss_value : 0.4542310949166616 
2021-10-12 19:57:30 - INFO - __call__ - 72 : EmbeddingSimilarityEvaluator: Evaluating the model on  dataset in epoch 2 after 565 steps:


2021-10-12 19:57:57 - INFO - __call__ - 103 : Cosine-Similarity :	Pearson: 0.8900	Spearman: 0.6423
2021-10-12 19:57:57 - INFO - __call__ - 105 : Manhattan-Distance:	Pearson: 0.8943	Spearman: 0.6422
2021-10-12 19:57:57 - INFO - __call__ - 107 : Euclidean-Distance:	Pearson: 0.8952	Spearman: 0.6422
2021-10-12 19:57:57 - INFO - __call__ - 109 : Dot-Product-Similarity:	Pearson: 0.8742	Spearman: 0.6407
2021-10-12 19:57:57 - INFO - save - 371 : Save model to /data/nfs14/nfs/aisearch/asr/xhsun/bwbd_recall/tmp


2021-10-12 19:57:58 - INFO - train - 98 : In epoch 2, training_step 1692, the eval score is 0.642281198590041, previous eval score is 0.6422547554728177, model has been saved in /data/nfs14/nfs/aisearch/asr/xhsun/bwbd_recall/tmp
2021-10-12 20:01:13 - INFO - train - 75 : Epoch : 2, train_step : 2025/11290, loss_value : 0.458108145793279 


KeyboardInterrupt: 

In [35]:
evaluator(model=simcse)

2021-10-12 20:03:49 - INFO - __call__ - 72 : EmbeddingSimilarityEvaluator: Evaluating the model on  dataset:


2021-10-12 20:04:16 - INFO - __call__ - 103 : Cosine-Similarity :	Pearson: 0.8917	Spearman: 0.6424
2021-10-12 20:04:16 - INFO - __call__ - 105 : Manhattan-Distance:	Pearson: 0.8935	Spearman: 0.6421
2021-10-12 20:04:16 - INFO - __call__ - 107 : Euclidean-Distance:	Pearson: 0.8943	Spearman: 0.6421
2021-10-12 20:04:16 - INFO - __call__ - 109 : Dot-Product-Similarity:	Pearson: 0.8776	Spearman: 0.6408


0.6423636731804672